In [1]:
import warnings;
warnings.simplefilter('ignore')

In [2]:
!pip install prophet
!conda install -c conda-forge fbprophet -y
!pip install --upgrade plotly

Solving environment: ...working... done

# All requested packages already installed.



In [3]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from prophet import Prophet
from fbprophet import Prophet

In [21]:
df = pd.read_csv("sqllab_untitled_query_1_20220712T111008.csv")
df.dropna(axis=1)

,country_name,country_code,region,year,SP_POP_GROW,SP_POP_TOTL
0,Afghanistan,AFG,South Asia,1960-01-01,1.813677,8994793.0
1,Afghanistan,AFG,South Asia,1961-01-01,1.874003,9164945.0
2,Afghanistan,AFG,South Asia,1962-01-01,1.932414,9343772.0
3,Afghanistan,AFG,South Asia,1963-01-01,1.989785,9531555.0
4,Afghanistan,AFG,South Asia,1964-01-01,2.046675,9728645.0
...,...,...,...,...,...,...
995,Bahrain,BHR,Middle East & North Africa,1980-01-01,4.859999,359902.0
996,Bahrain,BHR,Middle East & North Africa,1981-01-01,3.875819,374125.0
997,Turks and Caicos Islands,TCA,Latin America & Caribbean,2002-01-01,7.416459,21740.0
998,Bahrain,BHR,Middle East & North Africa,1982-01-01,3.111786,385950.0


In [25]:
df.head()

,country_name,country_code,region,year,NY_GNP_PCAP_CD,SE_ADT_1524_LT_FM_ZS,SE_ADT_1524_LT_MA_ZS,SE_ADT_1524_LT_ZS,SE_ADT_LITR_FE_ZS,SE_ADT_LITR_MA_ZS,...,SP_REG_BRTH_UR_ZS,SP_REG_BRTH_ZS,SP_REG_DTHS_ZS,SP_RUR_TOTL,SP_RUR_TOTL_ZG,SP_RUR_TOTL_ZS,SP_URB_GROW,SP_URB_TOTL,SP_URB_TOTL_IN_ZS,SP_UWT_TFRT
0,Afghanistan,AFG,South Asia,1960-01-01,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8255331.0,1.511229,91.779,5.253807,739462.0,8.221,NaN
1,Afghanistan,AFG,South Asia,1961-01-01,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8385191.0,1.560800,91.492,5.305559,779754.0,8.508,NaN
2,Afghanistan,AFG,South Asia,1962-01-01,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8521053.0,1.607275,91.195,5.363622,822719.0,8.805,NaN
3,Afghanistan,AFG,South Asia,1963-01-01,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8663230.0,1.654771,90.890,5.395136,868325.0,9.110,NaN
4,Afghanistan,AFG,South Asia,1964-01-01,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8811623.0,1.698401,90.574,5.456539,917022.0,9.426,NaN


In [19]:
#returns description of the data in the DataFrame, specifcally numerical data
df.describe()

,NY_GNP_PCAP_CD,SE_ADT_1524_LT_FM_ZS,SE_ADT_1524_LT_MA_ZS,SE_ADT_1524_LT_ZS,SE_ADT_LITR_FE_ZS,SE_ADT_LITR_MA_ZS,SE_ADT_LITR_ZS,SE_ENR_ORPH,SE_PRM_CMPT_FE_ZS,SE_PRM_CMPT_MA_ZS,...,SP_REG_BRTH_UR_ZS,SP_REG_BRTH_ZS,SP_REG_DTHS_ZS,SP_RUR_TOTL,SP_RUR_TOTL_ZG,SP_RUR_TOTL_ZS,SP_URB_GROW,SP_URB_TOTL,SP_URB_TOTL_IN_ZS,SP_UWT_TFRT
count,543.000000,31.000000,31.000000,32.000000,33.000000,33.000000,33.000000,2.000000,186.000000,186.000000,...,10.000000,17.000000,44.000000,9.970000e+02,969.000000,997.000000,997.000000,9.970000e+02,997.000000,8.00000
mean,11675.690608,0.931672,94.135052,92.100085,85.050803,91.083078,88.114461,0.875000,86.097776,89.405835,...,79.500000,85.270588,88.043182,2.633893e+06,0.528001,38.385514,2.235182,4.015998e+06,61.614486,12.51250
std,16665.260236,0.167382,12.119576,16.311477,25.655099,16.373567,20.804217,0.035355,25.200458,17.811963,...,31.440031,28.914070,12.764092,4.129796e+06,2.257552,23.961536,2.292630,7.425903e+06,23.961536,5.76899
min,180.000000,0.243310,45.796020,30.066350,4.987460,30.307750,18.157680,0.850000,4.389990,18.379370,...,11.500000,6.000000,53.000000,0.000000e+00,-8.884243,0.000000,-5.027368,5.331000e+03,8.221000,1.30000
25%,1990.000000,0.969948,95.420349,95.606823,93.580890,94.263390,93.912860,0.862500,81.832588,84.121032,...,68.875000,93.600000,79.525000,1.634700e+04,-0.571960,16.699000,0.647413,3.595000e+04,40.926000,10.65000
50%,5150.000000,0.999890,98.965729,99.082413,97.219910,98.008163,97.344160,0.875000,94.664720,93.867138,...,96.550000,98.600000,93.000000,6.588100e+04,0.609019,34.434000,1.825981,3.171730e+05,65.566000,13.20000
75%,15740.000000,1.001285,99.889854,99.892062,99.419998,99.672226,99.523964,0.887500,100.803101,99.509897,...,98.525000,99.600000,99.900000,3.103593e+06,1.753309,59.074000,3.597366,4.592914e+06,83.301000,16.07500
max,120490.000000,1.008740,100.000000,100.000000,99.719658,99.865921,99.789360,0.900000,138.003220,119.515671,...,99.600000,100.000000,100.000000,2.331516e+07,10.188589,91.779000,11.175584,3.937142e+07,100.000000,19.30000
